## Download Dataset and Unzip it

In [2]:
!unzip "df.zip"

Archive:  df.zip
  inflating: df.csv                  


## Import Librairies

In [3]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_acf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Preparation

1. read the csv, parse dates and set the index column to the date column (can all be in one function call)
2. groupy the dataframe by year and month and sum
3. set the column name to "value"

In [ ]:
### FIX ME
df = pd.read_csv("df.csv")
### !FIX ME

In [ ]:
df.plot()

In [ ]:
plot_acf(df["value"].dropna(), lags=50)

## Making the data stationary

In order our models to work, we need to make sure our data is stationary.

Here are the steps that you will need to follow:
- compute the log of the data
- differencing to get month to month variation
- differencing again to remove seasonality (approx. 12 months lag)

Again, those steps can be completed with only one function call using pandas.DataFrame features.

https://pandas.pydata.org/docs/

In [ ]:
def check_stationarity(series):
    result = adfuller(series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic', 'p-value', 'Number of Lags Used', 'Number of Observations Used']
    for value, label in zip(result, labels):
        print(f'{label} : {value}')
    if result[1] <= 0.05:
        print("Data is stationary")
    else:
        print("Data is not stationary")

In [ ]:
### FIX ME
##
### !FIX ME

In [ ]:
check_stationarity(df_testing["value"].dropna())

Augmented Dickey-Fuller Test:
ADF Test Statistic : -5.225226609716364
p-value : 7.807176920050943e-06
Number of Lags Used : 3
Number of Observations Used : 55
Data is stationary


In [ ]:
###
# Here you can plot the autocorrelation diagram again to see the difference
###

## Computing the AR(p) model

We need a function that takes a value of p (remember the slides) and a dataframe.

It should return a tuple containing: (rmse, p, theta, intercept of the regression, the dataframe with prediction included)

In [ ]:
def ar(p, df):
    """
    To make things easier, you will need to create as many columns as necessary to perform the regression.
    If the autoregressive order (p) is 2, you will need to add 2 columns, one with a shift of 1, and the other with a shift
    of 2.

    You then split the dataset into 2 parts: train and test (80% for train and the rest for test).

    Then you will drop the nan values, because this won't work with the regression.

    You will train the linear regression, get the coefficients and the intercept.

    Then create a new column "prediction" in your train and test datasets that will
    be filled with the predicted values.

    You compute the RMSE. You can use the mean_squared_error import above not to waste time on it.
    Of course you can write your own.

    Return [rmse, p, coefficients of the regression, intercept, pd.concat([train, test])]

    The sets need to be returned because we are using it later.
    """
    data = df.copy()
    ###
    # FIX ME
    ###
    return [rmse, p, lr.coef_.T, lr.intercept_, pd.concat([train, test])]

In [ ]:
best_rmse = 100000000
chosen_p  = -1

### FIX ME
# Iterate on values of p and take the best (lower RMSE).
### !FIX ME

In [ ]:
[rmse, _, theta, intercept, result_prediction_df] = ar(chosen_p, pd.DataFrame(df_testing["value"]))

In [ ]:
result_prediction_df[["value", "prediction"]].plot()

## Computing the MA model

In [ ]:
def ma(q, df):
    """
    The MA function is extremely similar to the AR function.

    It is another linear regression but on residual data instead.
    """
    data = df.copy()
    ###
    # FIX ME
    ###
    return [rmse, q, lr.coef_.T, lr.intercept_, pd.concat([train, test])]

In [ ]:
### FIX ME
# Here you generate the residuals data by substracting the AR(p) prediction to the true value
### !FIX ME


residuals = pd.DataFrame()
residuals["residual"] = result_prediction_df["value"] - result_prediction_df["prediction"]

In [ ]:
# The following function will display the distribution of the residual

residuals.plot(kind="kde")

In [ ]:
best_rmse = 100000000
chosen_q  = -1

### FIX ME
# Again iterate on values of q and take the best (lower RMSE).
### !FIX ME

In [ ]:
[rmse, _, theta, intercept, result_residuals_df] = ma(chosen_q, pd.DataFrame(residuals["residual"]))

In [ ]:
result_residuals_df[["residual", "prediction"]].plot()

## Forecasting

In [ ]:
result_prediction_df["prediction"] += result_residuals_df["prediction"]
result_prediction_df[["value", "prediction"]].plot()

## Back to the original scale

We simply backtrack the operations we performed to make the data stationary so we get back to the original scale.

In [ ]:
results = result_prediction_df.copy()
results["value"] += np.log(df).shift(1)["value"]
results["value"] += np.log(df).diff().shift(12)["value"]
results["value"] = np.exp(results["value"])
results["prediction"] += np.log(df).shift(1)["value"]
results["prediction"] += np.log(df).diff().shift(12)["value"]
results["prediction"] = np.exp(results["prediction"])

In [ ]:
results[["value", "prediction"]].plot()